<a href="https://colab.research.google.com/github/hennyzop18/Maths4ML/blob/GiuaKi/Bai2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def load_mnist():
    """Load dữ liệu MNIST từ file npz."""
    with np.load('/mnist.npz') as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
    return (x_train, y_train), (x_test, y_test)

def sigmoid(x):
    """Hàm kích hoạt sigmoid."""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Đạo hàm của hàm sigmoid."""
    return x * (1 - x)

def to_categorical(y, num_classes):
    """Chuyển nhãn số thành one-hot vector."""
    return np.eye(num_classes)[y]

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Khởi tạo trọng số
        self.w1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.w2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        """Lan truyền xuôi."""
        self.z1 = np.dot(X, self.w1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output, learning_rate):
        """Lan truyền ngược và cập nhật trọng số."""
        self.output_error = y - output
        self.output_delta = self.output_error * sigmoid_derivative(output)

        self.hidden_error = np.dot(self.output_delta, self.w2.T)
        self.hidden_delta = self.hidden_error * sigmoid_derivative(self.a1)

        # Cập nhật trọng số
        self.w2 += learning_rate * np.dot(self.a1.T, self.output_delta)
        self.b2 += learning_rate * np.sum(self.output_delta, axis=0, keepdims=True)
        self.w1 += learning_rate * np.dot(X.T, self.hidden_delta)
        self.b1 += learning_rate * np.sum(self.hidden_delta, axis=0, keepdims=True)

    def train(self, X, y, epochs, learning_rate, batch_size):
        """Huấn luyện mạng neural."""
        for epoch in range(epochs):
            total_loss = 0
            # Xử lý theo batch
            for i in range(0, len(X), batch_size):
                batch_X = X[i:i + batch_size]
                batch_y = y[i:i + batch_size]

                # Forward pass
                output = self.forward(batch_X)

                # Tính loss
                loss = np.mean(np.square(batch_y - output))
                total_loss += loss

                # Backward pass
                self.backward(batch_X, batch_y, output, learning_rate)

            if epoch % 5 == 0:
                print(f'Epoch {epoch}, Loss: {total_loss/len(X):.4f}')

    def predict(self, X):
        """Dự đoán lớp cho dữ liệu đầu vào."""
        output = self.forward(X)
        return np.argmax(output, axis=1)

# Chuẩn bị dữ liệu
(x_train, y_train), (x_test, y_test) = load_mnist()

# Tiền xử lý dữ liệu
X_train = x_train.reshape(x_train.shape[0], -1) / 255.0  # Chuẩn hóa và làm phẳng dữ liệu
X_test = x_test.reshape(x_test.shape[0], -1) / 255.0
y_train_cat = to_categorical(y_train, 10)

# Khởi tạo và huấn luyện mạng
input_size = 784  # 28x28 pixels
hidden_size = 128
output_size = 10  # 10 chữ số

nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(X_train, y_train_cat, epochs=30, learning_rate=0.1, batch_size=32)

# Đánh giá mô hình
predictions = nn.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f'\nĐộ chính xác trên tập test: {accuracy:.4f}')

# Thử nghiệm dự đoán một số
test_idx = 0
test_image = X_test[test_idx:test_idx+1]
pred = nn.predict(test_image)[0]
print(f'\nSố thực tế: {y_test[test_idx]}')
print(f'Số dự đoán: {pred}')

Epoch 0, Loss: 0.0025
Epoch 5, Loss: 0.0002
Epoch 10, Loss: 0.0001
Epoch 15, Loss: 0.0001
Epoch 20, Loss: 0.0000
Epoch 25, Loss: 0.0000

Độ chính xác trên tập test: 0.9788

Số thực tế: 7
Số dự đoán: 7
